# CRUD con base de datos

In [7]:
#Importamos librerias

from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import mariadb

#creamos el objeto (alumno)

#Creacion de la interfaz (UI)

class alumno():
    def __init__(self,ventana):
        self.ventana=ventana
        self.ventana.title("Ingresar alumnos")
        
        marco=LabelFrame(self.ventana,text="Alumnos")
        marco.grid(row=0,column=0,columnspan=2,pady=20)
        
        Label(marco,text="Nombre").grid(row=0,column=0)
        self.nombre=Entry(marco)
        self.nombre.grid(row=0,column=1)
        self.nombre.focus()
        
        Label(marco,text="Clave").grid(row=1,column=0)
        self.clave=Entry(marco)
        self.clave.grid(row=1,column=1)
        
        
        self.crear=Button(marco,text="Crear Alumno",fg="White",bg="green",command=self.agregar)
        self.crear.grid(row=2,columnspan=2,sticky=W+E)
        self.crear["state"]="normal"
        
        self.editar=Button(marco,text="Editar Alumno",fg="black",bg="yellow",command=self.actualizar)
        self.editar.grid(row=3,columnspan=2,sticky=W+E)
        self.editar["state"]="disabled"
        
        self.borrar=Button(marco,text="Borrar Alumno",fg="black",bg="red",command=self.borrar)
        self.borrar.grid(row=4,columnspan=2,sticky=W+E)
        self.borrar["state"]="disabled"
        
        self.mensaje=Label(self.ventana,text="")
        self.mensaje.grid(row=1,columnspan=2,sticky=W+E,pady=5)
        
        Label(self.ventana,text="Buscar Nombre").grid(row=2,column=0)
        self.buscarnombre=Entry(self.ventana)
        self.buscarnombre.grid(row=2,column=1)
        
        Label(self.ventana,text="Buscar Clave").grid(row=3,column=0)
        self.buscarclave=Entry(self.ventana)
        self.buscarclave.grid(row=3,column=1)
        
        Label(self.ventana,text="").grid(row=4)
        
        self.buscaralumno=Button(self.ventana,text="Buscar Alumno",command=self.buscar)
        self.buscaralumno.grid(row=5,columnspan=2,sticky=W+E)
        
        self.cuadro=ttk.Treeview(self.ventana,columns=2)
        self.cuadro.grid(row=6,columnspan=2,sticky=W+E)
        self.cuadro.bind("<Double-Button-1>",self.dobleclick)
        self.cuadro.heading("#0",text="Nombre")
        self.cuadro.heading("#1",text="Clave")
        
#Creación de la experiencia (UX)

#Conexion con la base de datos (MYSQL)

    def queryalumno(self,query):
        try:
            conn=mariadb.connect(
            host="localhost",
            user="root",
            password="",
            database="escuela",
            autocommit=True)
        except Exception as err:
            self.mensaje["text"]="No fue posible conectarse a la base de datos"
            
        cur=conn.cursor()
        cur.execute(query)
        return cur
    
#Metodo mostrar datos

    def mostrar(self,where=""):
        registros=self.cuadro.get_children()
        for registro in registros:
            self.cuadro.delete(registro)
        cur=self.queryalumno("SELECT `nombre`,`clave` FROM `alumnos`")
        if len(where)>0:
            cur=self.queryalumno("SELECT `nombre`,`clave` FROM `alumnos`"+where)
            
        else:
            cur=self.queryalumno("SELECT `nombre`,`clave` FROM `alumnos`")
        for (nombre,clave) in cur:
            self.cuadro.insert('',0,text=nombre,values=clave)
            
#Metodo agregar datos

    def agregar(self):
        if(len(self.nombre.get())!=0 and len(self.clave.get())!=0):
            query=("INSERT INTO `alumnos` (`id`,`nombre`,`clave`) VALUES (NULL, '"+self.nombre.get()+"','"+self.clave.get()+"');")
            self.queryalumno(query)
            self.mensaje["text"]="El alumno se ha ingresado con exito"
            self.nombre.delete(0,END)
            self.clave.delete(0,END)
            self.nombre.focus()
        else:
            self.mensaje["text"]="No se ha podido agregar los datos"
            
        self.mostrar()
        
#Metodo actualizar datos

    def actualizar(self):
        if(len(self.nombre.get())!=0 and len(self.clave.get())!=0):
            query=("UPDATE alumnos set nombre='"+self.nombre.get()+"',clave='"+self.clave.get()+"' where clave='"+self.clavevieja+"';")
            self.queryalumno(query)
            self.mensaje["text"]="La actualizacion se realizo con exito"
            self.nombre.delete(0,END)
            self.clave.delete(0,END)
            self.nombre.focus()
            self.crear["state"]="normal"
            self.editar["state"]="disabled"
            self.borrar["state"]="disabled"
            
        else:
            self.mensaje["text"]="No fue posible actualizar los datos"
            
        self.mostrar()
        
#Metodo para doble click

    def dobleclick(self,event):
        self.clavevieja=str(self.cuadro.item(self.cuadro.selection())["values"][0])
        self.nombre.delete(0,END)
        self.clave.delete(0,END)
        self.nombre.focus()
        self.crear["state"]="disabled"
        self.editar["state"]="normal"
        self.borrar["state"]="normal"
        self.nombre.insert(0,str(self.cuadro.item(self.cuadro.selection())["text"]))
        self.clave.insert(0,str(self.cuadro.item(self.cuadro.selection())["values"][0]))
        
#Metodo para borrar alumno

    def borrar(self):
        if messagebox.askyesno(message="Realmente desea borrar los datos del alumnos?",title="Borrar Alumno")==True:
            query=("DELETE from alumnos where clave ='"+self.clavevieja+"';")
            self.queryalumno(query)
            self.nombre.delete(0,END)
            self.clave.delete(0,END)
            self.nombre.focus()
            self.mensaje["text"]="Se han eliminado con exito los datos del estudiante"
            self.crear["state"]="normal"
            self.editar["state"]="disabled"
            self.borrar["state"]="disabled"
            self.mostrar()
            
#Metodo para buscar datos

            
        
    def buscar(self):
        where="where 1=1"
        
        if (len(self.buscarnombre.get())>0):
            where=where+" and nombre='"+self.buscarnombre.get()+"'"
        if (len(self.buscarclave.get())>0):
            where=where+"and clave='"+self.buscarclave.get()+"'"
        self.mostrar(where)
        
        
        
if __name__=="__main__":
    ventana=Tk()
    aplicacion=alumno(ventana)
    aplicacion.mostrar()
    ventana.mainloop()




Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ALEJANDRO\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\ALEJANDRO\AppData\Local\Temp\ipykernel_12216\1931434959.py", line 171, in buscar
    self.mostrar(where)
  File "C:\Users\ALEJANDRO\AppData\Local\Temp\ipykernel_12216\1931434959.py", line 91, in mostrar
    cur=self.queryalumno("SELECT `nombre`,`clave` FROM `alumnos`"+where)
  File "C:\Users\ALEJANDRO\AppData\Local\Temp\ipykernel_12216\1931434959.py", line 80, in queryalumno
    cur.execute(query)
  File "C:\Users\ALEJANDRO\anaconda3\lib\site-packages\mariadb\cursors.py", line 312, in execute
    self._readresponse()
mariadb.ProgrammingError: You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'clave='550'' at line 1
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ALEJANDRO\ana

# Derivadas e integrales

In [1]:
# Programa que calcule la derivada y la integral de una función
from sympy import *
from sympy.parsing.sympy_parser import parse_expr # Leer función introducida
from tkinter import *

def derivada():
    try:
        x = symbols('x') #Declarar variable independiente
        fun_escrita = funcion.get()
        f = parse_expr(fun_escrita)
        derivada = diff(f,x)
        etiqueta.configure(text=derivada)
    except:
        etiqueta.configure(text="Introduce la función correctamente")
        
        
def integral():
    try:
        x = symbols('x') #Declarar variable independiente
        fun_escrita2 = funcion.get()
        g = parse_expr(fun_escrita2)
        integral = integrate(g,x)
        etiqueta.configure(text=integral)
    except:
        etiqueta.configure(text="Introduce la función correctamente")
    
ventana = Tk()
ventana.geometry('400x280')
ventana.title("Cálculo Diferencial e Integral: f(x)")

anuncio = Label(ventana, text="Introduce una función de x:", font=("Arial", 15), fg="blue")
anuncio.pack()

funcion = Entry(ventana, font=("Arial", 15))
funcion.pack()

etiqueta = Label(ventana, text="Resultado", font=("Arial", 15), fg="red")
etiqueta.pack()
 
boton1 = Button(ventana, text="Derivar Función", font=("Arial", 15), command=derivada)
boton1.pack()

boton2 = Button(ventana, text="Integrar Función", font=("Arial", 15), command=integral)
boton2.pack()

def quit(): #Función salir
    ventana.quit()     # detiene mainloop
    ventana.destroy()  # elimina la ventana de la memoria
                    

button3 = Button(master=ventana, text="Salir", font=("Arial", 15), command=quit)
button3.pack()

ventana.mainloop()